In [10]:
# 실습 #1: gradio의 chatbot interface를 사용하여 ChatGPT와 스트리밍하면서 대화하는 인터페이스를 구성하시오.

import gradio as gr
import os
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

# 싱글턴 GPT 응답 생성
def bot(history):
    # 사용자의 마지막 질문만 전달
    user_input = history[-1][0]
    messages = [
        {"role": "system", "content": "너는 대화형 챗봇이야. 상냥하게 대답해줘."},
        {"role": "user", "content": user_input}
    ]

    # 스트리밍 응답 생성
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )

    history[-1][1] = ""  # 봇 응답 초기화

    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta.content:
            history[-1][1] += delta.content
            time.sleep(0.02)
            yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')


/tmp/ipykernel_76129/160439823.py:49: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipykernel_76129/160439823.py:49: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://0.0.0.0:7876
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# 실습 #2: 만든 어플리케이션을 멀티턴(이전 대화 내용을 기억하고 문맥에 따라 응답하기 위함)으로 수정하시오.

import gradio as gr
import os
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

# GPT 응답 생성 및 스트리밍
def bot(history):
    # history를 ChatGPT가 이해할 수 있는 message 포맷으로 변환
    messages = [{"role": "system", "content": "너는 대화형 챗봇이야. 상냥하게 대답해줘."}]
    for user, bot_response in history:
        if isinstance(user, tuple):  # 파일 입력인 경우 무시
            continue
        messages.append({"role": "user", "content": user})
        if bot_response:
            messages.append({"role": "assistant", "content": bot_response})

    # 응답을 스트리밍 형태로 생성
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )

    history[-1][1] = ""  # 봇 응답 초기화

    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta.content:
            history[-1][1] += delta.content
            time.sleep(0.02)
            yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')
